### 0. Essential code snippets from clustering investigations - Setup 

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import pandas as pd
import numpy as np
df = pd.read_pickle(r'D:\Py_Prjs\OPA_repo\ClusterAnalysis\KMeans\Data\df_allInfo_clean.pkl')
df_numeric = df.select_dtypes(include=['float64'])
X = df_numeric.fillna(df_numeric.median())
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
X_scaled = pd.DataFrame(sc.fit_transform(X), columns=X.columns, index=X.index)

In [5]:
# k = 26/ knn
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
cluster = KMeans(n_clusters = 26, random_state=42)
cluster.fit(X_scaled)
c = cluster.cluster_centers_
l = cluster.labels_
df_26_cluster = pd.merge(pd.Series(X.index), pd.Series(l, name='cluster'), right_index=True, left_index=True)

### 1. Deciding the number of stocks to select
principles of selection (recursively to retain the initial cluster proportioning, without significantly changing the lower cluster count classes):
1. clusters with > mean number of tickers, ret = (cv/total tickers count)*(total tickers count - Sum tickercounts of small clusters)
2. clusters with <= mean number of tickers, retains their entire tickers
3. recursive mean based reduction of the cluster capacity to capture alternative combination of stock selction per cluster




In [6]:
c_vc = df_26_cluster.cluster.value_counts() 
c_vc

8     62
17    58
22    51
3     36
15    34
21    33
5     31
1     27
6     26
0     23
16    22
2     18
13    17
4     15
12     8
19     8
20     7
9      7
25     5
18     4
7      4
23     2
14     2
10     1
24     1
11     1
Name: cluster, dtype: int64

In [8]:
c_vc.loc[lambda x: x < c_vc.mean()]

2     18
13    17
4     15
12     8
19     8
20     7
9      7
25     5
18     4
7      4
23     2
14     2
10     1
24     1
11     1
Name: cluster, dtype: int64

In [9]:
c_vc[c_vc >= c_vc.mean()]

8     62
17    58
22    51
3     36
15    34
21    33
5     31
1     27
6     26
0     23
16    22
Name: cluster, dtype: int64

In [10]:
((c_vc[c_vc > c_vc.mean()]*(c_vc.sum() - c_vc[c_vc < c_vc.mean()].sum())/c_vc.sum()).round()).append(c_vc[c_vc < c_vc.mean()]).mean()

16.26923076923077

In [76]:
c_vc = df_26_cluster.cluster.value_counts() 
initial_mean = c_vc.mean()
df = pd.DataFrame([], index=c_vc.index)
i = 0
while True:
    df = pd.concat([df, pd.DataFrame({f'cluster_{i}': c_vc.values}, index=c_vc.index)], axis= 1)
    mean = c_vc.mean()
    c_vc = ((c_vc[c_vc > c_vc.mean()]*(c_vc.sum() - c_vc[c_vc < c_vc.mean()].sum())/c_vc.sum()).round()).append(c_vc[c_vc < c_vc.mean()])
    i = i + 1
    if c_vc.mean() == mean:
        df[f'cluster_{i}'] = c_vc.min()
        break
    

In [77]:
display(df)
display(pd.concat([df.sum(axis=0), df.mean(axis=0)], axis=1).rename(columns={0:'total count', 1:'mean'}))

,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8,cluster_9,cluster_10,cluster_11,cluster_12,cluster_13
8,62,50.0,42.0,35.0,24.0,19.0,13.0,11.0,9.0,8.0,4.0,3.0,2.0,1.0
17,58,46.0,39.0,32.0,22.0,18.0,12.0,10.0,9.0,8.0,4.0,3.0,2.0,1.0
22,51,41.0,35.0,29.0,20.0,16.0,11.0,9.0,8.0,7.0,3.0,3.0,2.0,1.0
3,36,29.0,25.0,21.0,14.0,11.0,8.0,7.0,6.0,5.0,2.0,2.0,2.0,1.0
15,34,27.0,23.0,19.0,13.0,10.0,7.0,6.0,5.0,4.0,4.0,3.0,2.0,1.0
21,33,26.0,22.0,18.0,12.0,10.0,7.0,6.0,5.0,4.0,4.0,3.0,2.0,1.0
5,31,25.0,21.0,17.0,12.0,10.0,7.0,6.0,5.0,4.0,4.0,3.0,2.0,1.0
1,27,22.0,19.0,16.0,11.0,9.0,6.0,6.0,5.0,4.0,4.0,3.0,2.0,1.0
6,26,21.0,18.0,15.0,10.0,8.0,8.0,7.0,6.0,5.0,2.0,2.0,2.0,1.0
0,23,18.0,15.0,12.0,12.0,10.0,7.0,6.0,5.0,4.0,4.0,3.0,2.0,1.0


,total count,mean
cluster_0,503.0,19.346154
cluster_1,423.0,16.269231
cluster_2,368.0,14.153846
cluster_3,314.0,12.076923
cluster_4,246.0,9.461538
cluster_5,209.0,8.038462
cluster_6,165.0,6.346154
cluster_7,145.0,5.576923
cluster_8,126.0,4.846154
cluster_9,107.0,4.115385


In [13]:

selected_cluster_cap = df[['cluster_0', 'cluster_5']]
selected_cluster_cap

,cluster_0,cluster_5
8,62,19.0
17,58,18.0
22,51,16.0
3,36,11.0
15,34,10.0
21,33,10.0
5,31,10.0
1,27,9.0
6,26,8.0
0,23,10.0


### 2. Ranking and selecting stocks before 'portfolio optimization' is applied
Now the task is to devise a ranking technique to select the number of stocks in column cluster_5 out of the cluster_1 stocks (for low cardinalities we dont have to change anything)

Algorithm:
1. Rank by feature priority
2. Filter out by setting a threshold x < beta < y [x and y can be input (or selection from a set of alternatives) to our model to determine the volitity the client is expected to absorb]


In [14]:
# analysis of beta in our data
X.beta.describe()

count    503.000000
mean       1.061084
std        0.416325
min        0.171702
25%        0.784303
50%        1.062499
75%        1.283739
max        3.657327
Name: beta, dtype: float64

In [15]:
df_26 = df_26_cluster.set_index('symbol')
df_26

,cluster
symbol,
A,8
AAL,7
AAP,3
AAPL,12
ABBV,21
...,...
LEN,3
CBOE,8
DLR,6


In [16]:
beta_clust = X[['beta']].join(df_26)
beta_clust[beta_clust.cluster == 4].sort_values('beta')
# low risk < 1, medium risk > 1 and < 2, high risk > 2 OR a manual beta interval

,beta,cluster
symbol,,
WRB,0.630387,4
BRO,0.734195,4
DHR,0.865512,4
MNST,0.903831,4
NDSN,0.925646,4
NDAQ,0.949712,4
STZ,0.984554,4
RJF,1.059985,4
GRMN,1.062499,4


In [40]:
X_clustered = X.join(df_26)

In [41]:
# selected features coming from hca.ipynb script. will have to combine and automate them
# beta_0 = 0
# beta_1 = 1
X_clustered_ranked = X_clustered  # [(X_clustered.beta >= beta_0) & (X_clustered.beta < beta_1)]
features_col = {'profitMargins': False,
                'operatingMargins': False,
                'fullTimeEmployees': False,
                'shortRatio': True,
                'heldPercentInstitutions': False,
                'pegRatio': False,
                'enterpriseValue': False,
                'sharesShort': True,
                'heldPercentInsiders': False,
                'sharesPercentSharesOut': True,
                'fiftyTwoWeekLow': True,
                'priceToSalesTrailing12Months': False,
                'twoHundredDayAverage': False,
                'trailingEps': False,
                'enterpriseToRevenue': False,
                'fiftyDayAverage': False,
                'revenuePerShare': False,
                'grossProfits': False,
                'sharesShortPriorMonth': True,
                'ebitdaMargins': False,
                'grossMargins': False}


In [42]:
w = 1
for k, v in features_col.items():
    X_clustered_ranked.sort_values(k, ascending=v, inplace=True)
    X_clustered_ranked[f'{k}_score'] = w
    X_clustered_ranked[f'{k}_score'] = X_clustered_ranked.groupby(['cluster'])[f'{k}_score'].cumsum()
    w = w + 0.01


In [43]:
X_clustered_ranked['overall_score'] = X_clustered_ranked.apply(lambda x: sum([x[f'{k}_score'] for k in features_col.keys()]), axis=1)

In [44]:
X_clustered_ranked['_rn'] = X_clustered_ranked.groupby('cluster')['overall_score'].rank()

In [92]:
X_clustered_ranked_wClusterCaps = X_clustered_ranked.reset_index().merge(df, left_on='cluster', right_on=df.index).set_index('symbol')

In [130]:
alternate_port = dict()
n = max([int(x.split('_')[-1] ) for x in list(df.columns)])
for i in range(n + 1):
    port = X_clustered_ranked_wClusterCaps[X_clustered_ranked_wClusterCaps['_rn'] <= X_clustered_ranked_wClusterCaps[f'cluster_{i}']]
    alternate_port[f'cluster_{i}'] = port

In [131]:
print('select from the alternative stock selections:')
for i in range(n + 1):
    print(f'----------------cluster_{i}-----------------')
    print('num stocks:', alternate_port[f'cluster_{i}'].shape[0])
    print('beta min:', alternate_port[f'cluster_{i}']['beta'].min())
    print('beta max:', alternate_port[f'cluster_{i}']['beta'].max())
    print('beta mean:', alternate_port[f'cluster_{i}']['beta'].mean())
    print('beta std:', alternate_port[f'cluster_{i}']['beta'].std())

select from the alternative stock selections:
----------------cluster_0-----------------
num stocks: 503
beta min: 0.171702
beta max: 3.657327
beta mean: 1.0610841838966205
beta std: 0.4163250345499588
----------------cluster_1-----------------
num stocks: 423
beta min: 0.171702
beta max: 3.657327
beta mean: 1.0494490815602835
beta std: 0.41203720052936155
----------------cluster_2-----------------
num stocks: 368
beta min: 0.171702
beta max: 2.813218
beta mean: 1.0411724266304347
beta std: 0.39020283882803025
----------------cluster_3-----------------
num stocks: 314
beta min: 0.171702
beta max: 2.813218
beta mean: 1.031960939490446
beta std: 0.39444053980030547
----------------cluster_4-----------------
num stocks: 246
beta min: 0.171702
beta max: 2.442528
beta mean: 1.0390320223577236
beta std: 0.38067516222648373
----------------cluster_5-----------------
num stocks: 209
beta min: 0.171702
beta max: 2.442528
beta mean: 1.0427677727272726
beta std: 0.38811079779833907
--------------

In [125]:
i = 5
alternate_port[f'cluster_{i}'][0].shape[0]

209